# Coursework 2: Cryptography

[x] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 2.1 ElGamal Public Key and Digital Signature (50%)

### Helper Function(s)

In [1]:
function pwr_mod(x::Int,p::Int,n::Int)
    #Compute  𝑥𝑛 ≡ 𝑎 mod𝑝
    # x is the number
    # p is the mod
    # n is the power

    # Initialize the "product"
    product = 1
    y = x
    # Break n to powers of 2 by expressing it as a vector of 'bits' using built in 'digits' function
    #e.g 789 = [1  0  1  0  1  0  0  0  1  1]
    for i in digits(n,base=2)
        # y = x^(2^iter) mod p, where iter is the iteration of the loop
        if i == 1
            # If the current power of the power of 2 of x is used 
            # to break down n, multiply "product" with it
            # Otherwise skip it 
            product = mod(product*y,p)
        end
        # Update y for the next iteration
        y = mod(y^2,p)
    end
    return product
end

pwr_mod (generic function with 1 method)

### 2.1.1 Key Generation

From [this list](https://primes.utm.edu/lists/small/10000.txt) choose a prime number $p$ that is larger than 10,000. Find a primitive element $b$ in $\mathbb{F}_p$ such that $b > \sqrt{p}$. Choose an integer $a^{\prime} > \sqrt{p}$ as your private key and use it to generate $a = b^{a^{\prime}} \in \mathbb{F}_p^*$. 

$(p,b,a)$ is the public information. 

In [2]:
p = 10067
b = 103 #do we need to calculate this?
a_prime = 107 #Using smaller numbers to aid runtime

a = pwr_mod(b, p, a_prime)
println("p = ",p)
println("b = ",b)
println("a = ",a)

p = 10067
b = 103
a = 7235


### 2.1.2 Encryption

Design and implement a function `ElGamal_Encypt`. Write necessary documentation. 

In [3]:
function ElGamal_Encrypt(p,b,a,msg)
    t = rand(1:100)
    print(t)
    e_msg = [pwr_mod(b, p, t), msg * pwr_mod(a, p, t)]
end
msg = rand(1:200)
e_msg = ElGamal_Encrypt(p,b,a,msg)

51

2-element Vector{Int64}:
   9417
 194619

### 2.1.3 Decryption

Design and implement a function `ElGamal_Decypt`. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your encryption and decryption functions. 

In [4]:
function ElGamal_Decrypt(p,b,a,e_msg)
    #m = ma^t(b^t)^(-a_prime)
    bt = e_msg[1]
    mat = e_msg[2]
    msg = 1 / pwr_mod(bt, p, a_prime)
    msg = mod(msg * mat,p)
end
println(msg == ElGamal_Decrypt(p,b,a,e_msg))

true


### 2.1.4 Digital Signature Sign

Design and implement a function `ElGamal_Sign` to digitally sign a message. Write necessary documentation. 

In [5]:
using LinearAlgebra
#Define random message
m = 82
#m = rand(1:200)
#print("MESSAGE: ", m, "\n")

#Choose random t between 1 and p-1
#gcd of t with p-1 must equal 1. gcd(t, p-1) = 1
t = 1551
#while gcd(t, p-1) != 1
    #t = rand(1:p-1)
#end

#calculate u where t*u = 1 mod(p-1)
u = 0
count = 0
loop = true

while loop == true
    if(mod((count * (p-1) + 1),t) == 0)
        u = (count * (p-1) + 1)/t
        loop = false
    end
    count = count + 1 
end
@show u

function ElGamal_Sign(m, p, b, t, a_prime)
    i = pwr_mod(b, p, t) 
    j = mod(u * (norm(m) - a_prime * norm(i)), p-1)
    
    return i, j
    
end

@show ElGamal_Sign(m, p, b, t, a_prime)

u = 9417.0
ElGamal_Sign(m, p, b, t, a_prime) = (3482, 2252.0)


(3482, 2252.0)

### 2.1.5 Digital Signature Verification

Design and implement a function `ElGamal_SignVerify` to verify the digital sigature. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your functions for digital signature and verification. 

In [6]:
function ElGamal_SignVerify(p, a, b, i, j, m)
    LHS = mod(pwr_mod(a, p, i) * pwr_mod(i, p, j),p)
    RHS = pwr_mod(b, p, m)
    @show LHS
    @show RHS
    return LHS == RHS
end

#i,j = ElGamal_Sign(m, p, b, t, a_prime)

j = 2252
i = 3482

ElGamal_SignVerify(p, a, b, i, j, m)

LHS = 5966
RHS = 5966


true

## 2.2 RSA Public Key and Digital Signature (50%)

### 2.2.1 Key Generation

From [this list](https://primes.utm.edu/lists/small/10000.txt) choose two prime numbers $p_1$ and $p_2$ that are larger than 10,000. Calculate $n=p_1 p_2$ and $t=(p_1-1)(p_2-1)$. Find an $e > 10,000$ such that $\text{gcd}(e,t)=1$. 

$(n,e)$ is the public key. 

First we create helper functions to generate random and valid keys

In [7]:
using Random
# Function to generate a random e for RSA public key
# Input: t
# Output: e
function generate_e(t::Int)
    # Initialize vector for the coprimes of t
    coprimes = []
    # Initialize x as 1
    x = 1
    # Initialize p
    p = 1
    while p < t
        # Append previous x to coprimes if it is not 1
        if x != 1
            append!(coprimes,x)
        end
        # Pick a random number x, 1 < x < t
        x = rand(2:t-1)
        # Divide x by the gcd(x,t) to find a coprime of t
        g = gcd(x,t)
        while g > 1
            x = div(x,g)
            g = gcd(x,t)
        end
        # Repeat to gather enough coprimes to create other coprimes of sizes up to t-1
        p = p*x
    end 
    k = rand(1:length(coprimes))
    return prod(coprimes[1:k])
end

# Function to generate a valid private RSA key
# Input: e,t
# Output: d
function generate_d(e::Int,t::Int)
    # Since gcd(e,t) = 1, from Bezout: a*e + b*t = 1 -> a*e = 1 - b*t
    # Notice that: ( 1 - b*t )mod(t) = 1 for b an integer
    # Hence we can set d (our private key) = a
    _,d,_ = gcdx(e,t)
    # Take d mod t since a can be negative
    return mod(d,t)
end

# Function to generate a valid key pair e and d
# Calls generate_e and generate_d
function RSA_key_pair(t::Int)
    e = generate_e(t)
    d = generate_d(e,t)
    return (d,e)
end

RSA_key_pair (generic function with 1 method)

In [8]:
# Initialize Random Primes
p1 = 21013
p2 = 70951

n = p1*p2
t = (p1-1)*(p2-1)
d , e = RSA_key_pair(t)

println("The public key (n,e) is: $n , $e")
println("The private key d is: $d")

The public key (n,e) is: 1490893363 , 1285966013
The private key d is: 346639277


### 2.2.2 Encryption

Design and implement a function `RSA_Encypt`. Write necessary documentation. 

In [9]:
# Function for RSA encryption
# Input:  m, the message in a string format
#         n, n from RSA public key, must be greater than 128
#         e, e from RSA public key
# Output: c, vector of integers of the encrypted characters in m
function RSA_Encypt(m::String,n::Int,e::Int)
    # Convert the characters of m to integers using the ascii values
    m_ascii = [ Int(i) for i in m]
    # Encryption
    m_encry = [ pwr_mod(i,n,e) for i in m_ascii]
    return m_encry
end

RSA_Encypt (generic function with 1 method)

In [10]:
m = "This is my secret message"
c = RSA_Encypt(m,n,e)
println("Message m to be encrypted: '$m'")

Message m to be encrypted: 'This is my secret message'


### 2.2.3 Decryption

Design and implement a function `RSA_Decypt`. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your encryption and decryption functions. 

In [11]:
# Function for RSA encryption
# Input:  c, encrypted message, vector of integers
#         n, n from RSA public key, must be greater than 128 to support ascii
#         d, private key
# Output: m, decrypted message
function RSA_Decypt(c::Vector{Int64},n::Int,d::Int)
    m_ascii = [ pwr_mod(i,n,d) for i in c]
    # Convert the integers back to characters
    m = [ Char(i) for i in m_ascii]
    # Return the string
    return String(m)
end

RSA_Decypt (generic function with 1 method)

In [12]:
m_decry = RSA_Decypt(c,n,d)
println("Decrypted message: $m_decry")
print("Is it the same as the original message? ")
print(m_decry == m)

Decrypted message: This is my secret message
Is it the same as the original message? true

### 2.2.4 Digital Signature Sign

Design and implement a function `RSA_Sign` to digitally sign a message. Write necessary documentation. 

In [13]:
# Function for RSA Signature Creation
# Input:  m, the message in a string format
#         n, n from RSA public key, must be greater than 128
#         d, private key
#         f, hash function for m (Optional, default is sum)
# Output: s, s is the hash of the message m for RSA signature
#         y, is the signature for RSA signature
function RSA_Sign(m::String,n::Int,d::Int;f::Function=sum)
    # the function s is chosen to be the sum of the elements in the message
    s = f([Int(i) for i in m])
    y = pwr_mod(s,n,d)
    return s,y
end

RSA_Sign (generic function with 1 method)

### 2.2.5 Digital Signature Verification

Design and implement a function `RSA_SignVerify` to verify the digital sigature. Write necessary documentation. 

Use at least 3 examples to demonstrate the correctness of your functions for digital signature and verification. 

In [14]:
# Function for RSA Signature Verification
# Input:  m, the message in a string format
#         n, n from RSA public key, must be greater than 128
#         d, private key
# Output: bool, check whether the singature is valid
function RSA_SignVerify(s::Int,y::Int,e::Int,n::Int)
    return pwr_mod(y,n,e) == s
end

RSA_SignVerify (generic function with 1 method)

In [15]:
# Signature generation
# Signature 1: generated with the original message
s1 , y1 = RSA_Sign(m,n,d)

# Signature 2: generated with a new message and different hash function
new_m = "New message"
h(x) = trunc(Int,sqrt((sum(x)*2)))
s2 , y2 = RSA_Sign(new_m,n,d;f=h)

# Signature 3: generated with a false private key
false_d = rand(2:t-1)
s3 , y3 = RSA_Sign(m,n,false_d)

# Signature Verification
c1 = RSA_SignVerify(s1,y1,e,n)
c2 = RSA_SignVerify(s2,y2,e,n) 
c3 = RSA_SignVerify(s3,y3,e,n) 

println("Is Signature 1 valid (Must be true): $c1")
println("Is Signature 2 valid (Must be true): $c2")
println("Is Signature 3 valid (Must be false): $c3")

Is Signature 1 valid (Must be true): true
Is Signature 2 valid (Must be true): true
Is Signature 3 valid (Must be false): false


## Highlight

Please list a couple of highlights of your coursework that may impress your markers.

2.2.1 Created functions to generate valid and random RSA private and public keys

2.2.2 The function can encrypt anything in a string format

2.2.4 The function can be configured to have a custom hash function